In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, datePublished, url, full_text FROM nlp_articles WHERE datePublished<'2021-01-01' ", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

In [ ]:
from haystack.nodes import FARMReader,  TransformersReader
from haystack.utils import print_answers

In [ ]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [ ]:
dicts = [{"content": row["full_text"], "meta": {"name": row["url"]}} for i, row in df.iterrows()]

In [ ]:
document_store.write_documents(dicts)

In [ ]:
from haystack.nodes import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

In [ ]:
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader = FARMReader(model_name_or_path="Sahajtomar/German-question-answer-Electra", use_gpu=True)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(query="Wer ist der Chef von Apple?") 
print_answers(prediction, details="minimal")

In [ ]:
prediction = pipe.run(query="Wie viel verdient Tim Cook?")
print_answers(prediction, details="minimal")

In [ ]:
prediction = pipe.run(query="Wie hoch ist der Gewinn von Google?")
print_answers(prediction, details="minimal")

In [ ]:
prediction = pipe.run(query="Was ist Corona?")
print_answers(prediction, details="minimal")

In [ ]:
prediction = pipe.run(query="Was ist das wichtigste Produkt von Microsoft?")
print_answers(prediction, details="minimal")